In [1]:
import tensorflow as tf
import numpy as np
import cv2
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,Input,ReLU,Activation,Concatenate,BatchNormalization,Conv2DTranspose
import os
from matplotlib import pyplot

C:\Users\Aryan\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Aryan\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Aryan\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

encoding (256 X 256 X 3) to (32 X 32 X 256)

In [3]:
class CompressX:
    @staticmethod
    def encoder(im_shp,channels):
        first = Input(shape = (im_shp,im_shp,channels))
        d = Conv2D(64,(3,3),padding='same')(first)
        d = ReLU()(d)
        d = Conv2D(128,(3,3),strides=(2,2),padding='same')(d)
        d = ReLU()(d)
        d = Conv2D(256,(3,3),strides=(2,2),padding='same')(d)
        d = ReLU()(d)
        d = Conv2D(512,(3,3),strides=(2,2),padding='same')(d)
        last = ReLU()(d)
        
        model = Model(first,last)
        return model
    
    @staticmethod
    def decoder(im_shp,channels):
        first = Input(shape = (im_shp,im_shp,channels))
        d = Conv2DTranspose(256,(3,3),strides=(2,2),padding='same')(first)
        d = ReLU()(d)
        d = Conv2DTranspose(128,(3,3),strides=(2,2),padding='same')(d)
        d = ReLU()(d)
        d = Conv2DTranspose(64,(3,3),strides=(2,2),padding='same')(d)
        d = ReLU()(d)
        d = Conv2DTranspose(3,(3,3),padding='same')(d)
        last = ReLU()(d)
        
        
        model = Model(first,last)
        return model
    
    @staticmethod
    def comp(encoder,decoder,im_shp,channels):
        rgb_inp = Input(shape=(im_shp,im_shp,channels))
        rgb_en_out = encoder(rgb_inp)
        rgb_de_out = decoder(rgb_en_out)
        
        hsv_inp = Input(shape=(im_shp,im_shp,channels))
        hsv_en_out = encoder(hsv_inp)
        hsv_de_out = decoder(hsv_en_out)
        
        hsv2rgb = Conv2D(3,(3,3),padding='same')(hsv_de_out)
     
        
        model = Model([rgb_inp,hsv_inp],[rgb_de_out,hsv_de_out,hsv2rgb])
        opt = Adam(lr=0.0002)
        model.compile(loss=['mse','mse','mse'],optimizer=opt)
        return model
    
    @staticmethod
    def load_ds(n,step,path):
        names = os.listdir(path)
        strt = step*n
        end = strt + n
        return names[strt:end]
    
    @staticmethod
    def load_rgb_samples(n,step):
        loaded = []
        image_names = CompressX.load_ds(n,step,'ds/rgb')
        for image in image_names:
            img_path = os.path.join('ds/rgb',image)
            img = cv2.imread(img_path)
            img = (img - 127.5)/127.5
            loaded.append(img)
        return loaded
    
    @staticmethod
    def load_hsv_samples(n,step):
        loaded = []
        image_names = CompressX.load_ds(n,step,'ds/hsv')
        for image in image_names:
            img_path = os.path.join('ds/hsv',image)
            img = cv2.imread(img_path)
            loaded.append(img)
        return loaded
    
    @staticmethod
    def train(comp,EPOCHS,batch_size=60):
        batches_per_epoch = int(13200/batch_size)
        for epoch in EPOCHS:
            print(f'[INFO] Starting epoch {epoch} of {EPOCHS}')
            for batch_no in batches_per_epoch:
                hsv = CompressX.load_hsv_samples(batch_size,batch_no)
                rgb = CompressX.load_rgb_samples(batch_size,batch_no)
                l1,l2,l3 = comp.train_on_batch([rgb,hsv],[rgb,hsv,rgb])
                
                if batch_no %20 == 0:
                    print(f'[INFO] Ended step {batch_no} of {batches_per_epoch}')
                    print(f'[INFO] l1={l1}  ,  l2={l2}  ,  l3={l2}')
                    result = comp.predict(rgb[0])
                    result = (result+1)*127.5
                    im_name = f'output/{epoch}_{batch_no}.jpg'
                    cv2.imwrite(im_name,result)
                    
                if batch_no%100 == 0:
                    comp.save('saved_models/')
                    
                    
        

In [4]:
# img = CompressX.load_rgb_samples(10,0)[0]
# print(np.amax(img))
# cv2.imshow('dsfds',img)
# cv2.waitKey(0)
